# Домашнее задание

## Реализация градиентного бустинга

В рамках этой задачи нужно написать градиентный бустинг над решающими деревьями в задаче классификации. В качестве функции потерь предлагается взять **log loss**. Про него можно прочитать подробнее здесь: https://scikit-learn.org/stable/modules/model_evaluation.html#log-loss


$y_i$ это правильный ответ (0 или 1), $\hat{y}_i$ это ваше предскзаание

Может показаться, что надо максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$, где $y_i$

Да, но нет. Лучше максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(f(\hat{y}_i)) + (1 - y_i) \log(1 - f(\hat{y}_i))$, где $f(x) = \frac{1}{1 + e^{-x}}$. Благодаря этому у вас не будет ограничений на принимаеммые значения для $\hat{y}_i$

### Задание 1

Напишите вычисление производной f(x), обычно её называют **сигмоида**.

In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))


def der_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

In [3]:
der_sigmoid(0) == 0.25

True

In [4]:
der_sigmoid(np.array([0, 0])) == np.array([0.25, 0.25])

array([ True,  True])

In [5]:
der_sigmoid(np.log(3)) == 0.1875

True

**Значение для формы:**

In [6]:
print(round(der_sigmoid(np.array([-10, 4.1, -1, 2])).sum() + sigmoid(0.42), 4))

0.9212


Хорошо, теперь мы умеем считать производную функции f, но надо найти производную log loss-а по $\hat{y}$ в первом варианте 

Напоминание, первый вариант это  $y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$

### Задание 2

Напишите вычисление производной log loss-a

In [7]:
def der_log_loss(y_hat, y_true):
    """
    0 < y_hat < 1
    """
    return y_true*(1/y_hat) - (1-y_true)/(1-y_hat)

In [8]:
der_log_loss(0.5, 0) == -2

True

In [9]:
der_log_loss(0.5, 1) == 2

True

In [10]:
der_log_loss(np.array([0.8, 0.8]), np.array([1, 1])) == np.array([1.25, 1.25])

array([ True,  True])

**Значение для формы**

In [11]:
print(round(-sum(der_log_loss((x + 1) / 100., x % 2) for x in range(99)), 2))

69.82


Отлично теперь мы можем воспользоваться производной сложной функции и получить вычисление градиента формулы по второму варианту:

In [12]:
def calc_gradient(y_hat, y_true):
    return der_log_loss(sigmoid(y_hat), y_true) * der_sigmoid(y_hat)

Теперь мы можем написать код градиентного бустинга для классификации

### Задание 3

Допишите класс

In [13]:
from sklearn.base import BaseEstimator # чтобы поддержать интерфейс sklearn
from sklearn.tree import DecisionTreeRegressor # для обученияb на каждой итерации

In [14]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        """
        tree_params_dict - словарь параметров, которые надо использовать при обучении дерева на итерации
        iters - количество итераций
        tau - коэффициент перед предсказаниями деревьев на каждой итерации
        """
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        for iter_num in range(self.iters):
            # Нужно найти градиент функции потерь по предсказниям в точке curr_pred
            grad = calc_gradient(curr_pred, y_data)
            # Мы максимизируем, поэтому надо обучить DecisionTreeRegressor с параметрами 
            # tree_params_dict по X_data предсказывать grad
            algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data,grad)
            self.estimators.append(algo)
            # все предсказания домножаются на tau и обновляется переменная curr_pred
            curr_pred += self.tau * algo.predict(X_data)
    
    def predict(self, X_data):
        # изначально все предскзания нули
        res = np.zeros(X_data.shape[0])
        for estimator in self.estimators:
            # нужно сложить все предсказания деревьев с весом self.tau
            res+=estimator.predict(X_data)*self.tau
            
        return (res > 0).astype(int)

## Проверка качества полученного класса (в самом низу код для формы)

Можете поиграться с параметрами, посмотрим, у кого самое лучшее качество получится

In [15]:
# для оценки качества
from sklearn.model_selection import cross_val_score

# для генерации датасетов
from sklearn.datasets import make_classification

# для сравнения
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [16]:
X_data, y_data = make_classification(n_samples=1000, n_features=10, random_state=42)

In [35]:
tree_params_dict={'max_depth':4}
a=DecisionTreeRegressor(**tree_params_dict).fit(X_data[:5],y_data[:5])
b=[DecisionTreeRegressor(**tree_params_dict).fit(X_data[:5],y_data[:5]),DecisionTreeRegressor(**tree_params_dict).fit(X_data[25:55],y_data[25:55]),DecisionTreeRegressor(**tree_params_dict).fit(X_data[5:15],y_data[5:15])]

In [44]:
b[2].predict(X_data[5:10])

array([0., 0., 1., 1., 0.])

In [17]:
algo = SimpleGB(
    tree_params_dict={'max_depth':4},
    iters=100,
    tau = 0.1
)

In [18]:
tree_params_dict={'max_depth':4}

In [19]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='accuracy'))

0.921

In [20]:
np.mean(cross_val_score(DecisionTreeClassifier(**tree_params_dict), X_data, y_data, cv=5, scoring='accuracy'))

0.8910088502212556

In [21]:
np.mean(cross_val_score(XGBClassifier(**tree_params_dict,iters=1000,tau=0.1), X_data, y_data, cv=5, scoring='accuracy'))

0.9020041751043776

In [22]:
np.mean(cross_val_score(LogisticRegression(), X_data, y_data, cv=5, scoring='accuracy'))

0.8560087002175054

In [23]:
CatBoostClassifier?

Object `CatBoostClassifier` not found.


In [24]:
from catboost import CatBoostClassifier
np.mean(cross_val_score(CatBoostClassifier(**tree_params_dict,iterations=1000), X_data, y_data, cv=5, scoring='accuracy'))

Learning rate set to 0.015723
0:	learn: 0.6799840	total: 55.8ms	remaining: 55.7s
1:	learn: 0.6672044	total: 60.5ms	remaining: 30.2s
2:	learn: 0.6524810	total: 65.1ms	remaining: 21.6s
3:	learn: 0.6408636	total: 69.7ms	remaining: 17.4s
4:	learn: 0.6300696	total: 74.4ms	remaining: 14.8s
5:	learn: 0.6187209	total: 79.5ms	remaining: 13.2s
6:	learn: 0.6063977	total: 85.4ms	remaining: 12.1s
7:	learn: 0.5935315	total: 90ms	remaining: 11.2s
8:	learn: 0.5828629	total: 113ms	remaining: 12.4s
9:	learn: 0.5732389	total: 118ms	remaining: 11.7s
10:	learn: 0.5609633	total: 122ms	remaining: 11s
11:	learn: 0.5514437	total: 129ms	remaining: 10.6s
12:	learn: 0.5434696	total: 132ms	remaining: 10s
13:	learn: 0.5310925	total: 136ms	remaining: 9.59s
14:	learn: 0.5207290	total: 144ms	remaining: 9.47s
15:	learn: 0.5134199	total: 150ms	remaining: 9.21s
16:	learn: 0.5068012	total: 157ms	remaining: 9.08s
17:	learn: 0.4984458	total: 163ms	remaining: 8.87s
18:	learn: 0.4904296	total: 168ms	remaining: 8.68s
19:	learn

168:	learn: 0.2309920	total: 1.54s	remaining: 7.59s
169:	learn: 0.2305040	total: 1.56s	remaining: 7.6s
170:	learn: 0.2301478	total: 1.58s	remaining: 7.67s
171:	learn: 0.2290416	total: 1.61s	remaining: 7.74s
172:	learn: 0.2287793	total: 1.62s	remaining: 7.74s
173:	learn: 0.2284295	total: 1.62s	remaining: 7.71s
174:	learn: 0.2283371	total: 1.63s	remaining: 7.7s
175:	learn: 0.2280350	total: 1.67s	remaining: 7.81s
176:	learn: 0.2276260	total: 1.68s	remaining: 7.81s
177:	learn: 0.2271447	total: 1.69s	remaining: 7.79s
178:	learn: 0.2268664	total: 1.7s	remaining: 7.78s
179:	learn: 0.2261122	total: 1.7s	remaining: 7.76s
180:	learn: 0.2259436	total: 1.71s	remaining: 7.75s
181:	learn: 0.2256322	total: 1.72s	remaining: 7.72s
182:	learn: 0.2251288	total: 1.72s	remaining: 7.7s
183:	learn: 0.2249163	total: 1.73s	remaining: 7.68s
184:	learn: 0.2244816	total: 1.74s	remaining: 7.67s
185:	learn: 0.2234850	total: 1.75s	remaining: 7.66s
186:	learn: 0.2232607	total: 1.75s	remaining: 7.63s
187:	learn: 0.223

328:	learn: 0.1833239	total: 2.84s	remaining: 5.8s
329:	learn: 0.1832363	total: 2.85s	remaining: 5.79s
330:	learn: 0.1831022	total: 2.86s	remaining: 5.78s
331:	learn: 0.1828892	total: 2.87s	remaining: 5.77s
332:	learn: 0.1827595	total: 2.87s	remaining: 5.75s
333:	learn: 0.1822807	total: 2.88s	remaining: 5.74s
334:	learn: 0.1821229	total: 2.88s	remaining: 5.71s
335:	learn: 0.1817832	total: 2.89s	remaining: 5.71s
336:	learn: 0.1813845	total: 2.89s	remaining: 5.69s
337:	learn: 0.1811962	total: 2.89s	remaining: 5.67s
338:	learn: 0.1810482	total: 2.9s	remaining: 5.65s
339:	learn: 0.1808267	total: 2.9s	remaining: 5.63s
340:	learn: 0.1806095	total: 2.91s	remaining: 5.62s
341:	learn: 0.1803025	total: 2.91s	remaining: 5.6s
342:	learn: 0.1801085	total: 2.92s	remaining: 5.59s
343:	learn: 0.1800477	total: 2.92s	remaining: 5.57s
344:	learn: 0.1799929	total: 2.92s	remaining: 5.55s
345:	learn: 0.1797302	total: 2.93s	remaining: 5.54s
346:	learn: 0.1792212	total: 2.94s	remaining: 5.53s
347:	learn: 0.17

495:	learn: 0.1469462	total: 3.72s	remaining: 3.78s
496:	learn: 0.1469069	total: 3.72s	remaining: 3.77s
497:	learn: 0.1467500	total: 3.73s	remaining: 3.76s
498:	learn: 0.1466217	total: 3.73s	remaining: 3.75s
499:	learn: 0.1464094	total: 3.74s	remaining: 3.74s
500:	learn: 0.1462119	total: 3.75s	remaining: 3.73s
501:	learn: 0.1459741	total: 3.75s	remaining: 3.72s
502:	learn: 0.1457014	total: 3.76s	remaining: 3.71s
503:	learn: 0.1456607	total: 3.77s	remaining: 3.71s
504:	learn: 0.1456264	total: 3.77s	remaining: 3.7s
505:	learn: 0.1455320	total: 3.78s	remaining: 3.69s
506:	learn: 0.1454640	total: 3.78s	remaining: 3.68s
507:	learn: 0.1452336	total: 3.79s	remaining: 3.67s
508:	learn: 0.1449769	total: 3.79s	remaining: 3.66s
509:	learn: 0.1447996	total: 3.8s	remaining: 3.65s
510:	learn: 0.1445872	total: 3.8s	remaining: 3.64s
511:	learn: 0.1444150	total: 3.81s	remaining: 3.63s
512:	learn: 0.1443842	total: 3.82s	remaining: 3.63s
513:	learn: 0.1443143	total: 3.83s	remaining: 3.62s
514:	learn: 0.1

659:	learn: 0.1229655	total: 4.93s	remaining: 2.54s
660:	learn: 0.1227279	total: 4.93s	remaining: 2.53s
661:	learn: 0.1225320	total: 4.94s	remaining: 2.52s
662:	learn: 0.1223721	total: 4.94s	remaining: 2.51s
663:	learn: 0.1221973	total: 4.94s	remaining: 2.5s
664:	learn: 0.1219837	total: 4.95s	remaining: 2.49s
665:	learn: 0.1217164	total: 4.95s	remaining: 2.48s
666:	learn: 0.1214916	total: 4.96s	remaining: 2.47s
667:	learn: 0.1213837	total: 4.96s	remaining: 2.46s
668:	learn: 0.1212633	total: 4.96s	remaining: 2.46s
669:	learn: 0.1209777	total: 4.97s	remaining: 2.45s
670:	learn: 0.1208940	total: 4.97s	remaining: 2.44s
671:	learn: 0.1207123	total: 4.97s	remaining: 2.43s
672:	learn: 0.1205981	total: 4.98s	remaining: 2.42s
673:	learn: 0.1202572	total: 4.98s	remaining: 2.41s
674:	learn: 0.1201250	total: 4.99s	remaining: 2.4s
675:	learn: 0.1199429	total: 4.99s	remaining: 2.39s
676:	learn: 0.1197511	total: 4.99s	remaining: 2.38s
677:	learn: 0.1194856	total: 5s	remaining: 2.37s
678:	learn: 0.119

...

419:	learn: 0.1673296	total: 2.6s	remaining: 3.59s
420:	learn: 0.1666624	total: 2.61s	remaining: 3.59s
421:	learn: 0.1664372	total: 2.63s	remaining: 3.6s
422:	learn: 0.1662746	total: 2.63s	remaining: 3.59s
423:	learn: 0.1660193	total: 2.64s	remaining: 3.58s
424:	learn: 0.1658397	total: 2.64s	remaining: 3.57s
425:	learn: 0.1657094	total: 2.65s	remaining: 3.57s
426:	learn: 0.1654595	total: 2.65s	remaining: 3.56s
427:	learn: 0.1652229	total: 2.66s	remaining: 3.55s
428:	learn: 0.1650582	total: 2.66s	remaining: 3.54s
429:	learn: 0.1648776	total: 2.67s	remaining: 3.53s
430:	learn: 0.1646756	total: 2.67s	remaining: 3.53s
431:	learn: 0.1645631	total: 2.68s	remaining: 3.52s
432:	learn: 0.1642793	total: 2.68s	remaining: 3.51s
433:	learn: 0.1640881	total: 2.69s	remaining: 3.5s
434:	learn: 0.1640508	total: 2.69s	remaining: 3.5s
435:	learn: 0.1639971	total: 2.7s	remaining: 3.49s
436:	learn: 0.1639010	total: 2.7s	remaining: 3.48s
437:	learn: 0.1631991	total: 2.71s	remaining: 3.47s
438:	learn: 0.1630

586:	learn: 0.1373564	total: 3.6s	remaining: 2.53s
587:	learn: 0.1372019	total: 3.61s	remaining: 2.53s
588:	learn: 0.1370776	total: 3.61s	remaining: 2.52s
589:	learn: 0.1369854	total: 3.62s	remaining: 2.52s
590:	learn: 0.1368769	total: 3.63s	remaining: 2.51s
591:	learn: 0.1367645	total: 3.64s	remaining: 2.51s
592:	learn: 0.1363723	total: 3.65s	remaining: 2.51s
593:	learn: 0.1363273	total: 3.66s	remaining: 2.5s
594:	learn: 0.1362075	total: 3.67s	remaining: 2.5s
595:	learn: 0.1361470	total: 3.67s	remaining: 2.49s
596:	learn: 0.1357920	total: 3.68s	remaining: 2.48s
597:	learn: 0.1356585	total: 3.69s	remaining: 2.48s
598:	learn: 0.1354761	total: 3.69s	remaining: 2.47s
599:	learn: 0.1354350	total: 3.7s	remaining: 2.47s
600:	learn: 0.1352486	total: 3.71s	remaining: 2.46s
601:	learn: 0.1351270	total: 3.72s	remaining: 2.46s
602:	learn: 0.1350400	total: 3.72s	remaining: 2.45s
603:	learn: 0.1348896	total: 3.73s	remaining: 2.44s
604:	learn: 0.1346617	total: 3.73s	remaining: 2.44s
605:	learn: 0.13

746:	learn: 0.1161470	total: 4.59s	remaining: 1.56s
747:	learn: 0.1160685	total: 4.6s	remaining: 1.55s
748:	learn: 0.1160530	total: 4.61s	remaining: 1.54s
749:	learn: 0.1159296	total: 4.61s	remaining: 1.54s
750:	learn: 0.1157864	total: 4.62s	remaining: 1.53s
751:	learn: 0.1155527	total: 4.63s	remaining: 1.52s
752:	learn: 0.1154650	total: 4.63s	remaining: 1.52s
753:	learn: 0.1153405	total: 4.64s	remaining: 1.51s
754:	learn: 0.1153056	total: 4.65s	remaining: 1.51s
755:	learn: 0.1152296	total: 4.65s	remaining: 1.5s
756:	learn: 0.1151249	total: 4.66s	remaining: 1.5s
757:	learn: 0.1150500	total: 4.67s	remaining: 1.49s
758:	learn: 0.1149902	total: 4.67s	remaining: 1.48s
759:	learn: 0.1148238	total: 4.68s	remaining: 1.48s
760:	learn: 0.1147513	total: 4.69s	remaining: 1.47s
761:	learn: 0.1144404	total: 4.69s	remaining: 1.47s
762:	learn: 0.1143598	total: 4.7s	remaining: 1.46s
763:	learn: 0.1140600	total: 4.71s	remaining: 1.45s
764:	learn: 0.1139439	total: 4.71s	remaining: 1.45s
765:	learn: 0.11

918:	learn: 0.0980963	total: 5.72s	remaining: 504ms
919:	learn: 0.0979558	total: 5.73s	remaining: 498ms
920:	learn: 0.0977931	total: 5.74s	remaining: 492ms
921:	learn: 0.0977336	total: 5.74s	remaining: 486ms
922:	learn: 0.0975846	total: 5.75s	remaining: 480ms
923:	learn: 0.0974637	total: 5.76s	remaining: 474ms
924:	learn: 0.0974064	total: 5.76s	remaining: 467ms
925:	learn: 0.0973325	total: 5.77s	remaining: 461ms
926:	learn: 0.0971496	total: 5.77s	remaining: 455ms
927:	learn: 0.0970339	total: 5.78s	remaining: 448ms
928:	learn: 0.0967354	total: 5.78s	remaining: 442ms
929:	learn: 0.0966801	total: 5.79s	remaining: 436ms
930:	learn: 0.0965952	total: 5.8s	remaining: 430ms
931:	learn: 0.0964814	total: 5.81s	remaining: 424ms
932:	learn: 0.0964274	total: 5.81s	remaining: 418ms
933:	learn: 0.0963631	total: 5.82s	remaining: 411ms
934:	learn: 0.0962372	total: 5.82s	remaining: 405ms
935:	learn: 0.0961791	total: 5.83s	remaining: 399ms
936:	learn: 0.0959541	total: 5.84s	remaining: 392ms
937:	learn: 0

0.8929940748518714

**Значение для формы**

In [25]:
DecisionTreeRegressor?

In [26]:
from sklearn.model_selection import StratifiedKFold

print(round(np.mean(cross_val_score(SimpleGB(
    tree_params_dict={
        'max_depth': 4,'splitter':'best'
    },
    iters=1000,
    tau = 0.01
), X_data, y_data, cv=StratifiedKFold(4, random_state=42), scoring='accuracy')), 3))

0.923
